# 🔎 Análisis Exploratorio de Datos (Google):

## Importación de librerias

In [1]:
from textblob import TextBlob
import datetime
import inspect
import re
import matplotlib.pyplot as plt
import inspect
import pandas as pd
import gzip
import os
import seaborn as sns
import numpy as np
from data_utils import data_type_check

### Importación de los datasets

In [2]:
df_metadata = pd.read_parquet('../0_Dataset/Data_Limpia/Google/G_metadata_FL.parquet', engine='pyarrow')
df_review = pd.read_parquet('../0_Dataset/Data_Limpia/Google/G_review_FL_reducido.parquet', engine='pyarrow')

## Metadata de Florida (FL)

Verificación de los datos

In [12]:
df_metadata.head(5)

,name,address,latitude,longitude,category,avg_rating,num_of_reviews
0,Neurocare & Family Services,"Neurocare & Family Services, 1749 S Naperville...",41.839291,-88.100503,[Doctor],5.0,1
1,Bayview Canvas,"Bayview Canvas, 22460 Glass Ln, Port Charlotte...",26.962494,-82.075552,[Upholstery shop],5.0,1
2,Peluquería Acapulco Barbershop,"Peluquería Acapulco Barbershop, 1014 Gault Ave...",34.450636,-85.714130,[Barber shop],4.8,58
3,Mountain Travelers Hike and Ski Shop,"Mountain Travelers Hike and Ski Shop, 147 US-4...",43.622089,-72.953674,"[Ski shop, Camping store, Car accessories store]",4.4,26
4,Lovely Nails,"Lovely Nails, 636 Brandon Ave SW, Roanoke, VA ...",37.254535,-79.959470,[Nail salon],4.3,145


### 1. Información y transformación de los datos

In [ ]:
# SE REALIZARÁ UN REINICIO DE INDICES
df_metadata.reset_index(drop=True,inplace=True)

In [8]:
# OBTENEMOS SOLO LAS COLUMNAS QUE NOS VAN A AYUDAR A NUESTRO ANÁLISIS
df_metadata.drop(columns=['gmap_id','relative_results','url'],inplace=True)

In [22]:
# OBTENEMOS UN ANÁLISIS DE DATOS EN CUANTOS A LA CANTIDAD DE NULOS OBTENIDOS
data_type_check(df_metadata)


 Resumen del dataframe:

Dimensiones:  (212299, 7)
          columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0            name       100.0      0.0            0    object
1         address       100.0      0.0            0    object
2        latitude       100.0      0.0            0   float64
3       longitude       100.0      0.0            0   float64
4        category       100.0      0.0            0    object
5      avg_rating       100.0      0.0            0   float64
6  num_of_reviews       100.0      0.0            0     int64


#### 📚 Conclusión:
- Se ve que al realizar un análisis de cantidad de datos nulos, tenemos una gran cantidad de estos en las distintos registros y en las distintas variables del dataset.
- Nuestra norma en cuanto a la eliminación de nulos será si es mayor o igual al 18% se eliminarán, estas columnas serían: ['description', 'price', 'hours', 'MISC', 'state']
- Luego de eliminar las columnas procedemos a eliminar los registros que contienen registros nulos.

In [11]:
# ELIMINACIÓN DE COLUMNAS CON MAYOR CANTIDAD DE NULOS
df_metadata.drop(columns=['description', 'price', 'hours', 'MISC', 'state'] ,inplace=True)

In [20]:
# ELIMINACIÓN DE REGISTRO QUE CONTINENEN REGISTRO NULOS
df_metadata.dropna(axis=0,inplace=True)

### 2.Variables categóricas y cuantitativas

In [21]:
# VARIABLE CATEGÓRICAS Y CUANTITATIVAS EXISTENTES EN EL DATASET
df_metadata_cat = df_metadata.select_dtypes('O')
df_metadata_num = df_metadata.select_dtypes('number')

In [23]:
df_metadata_num

,latitude,longitude,avg_rating,num_of_reviews
0,41.839291,-88.100503,5.0,1
1,26.962494,-82.075552,5.0,1
2,34.450636,-85.714130,4.8,58
3,43.622089,-72.953674,4.4,26
4,37.254535,-79.959470,4.3,145
...,...,...,...,...
219996,38.408448,-121.447597,4.3,13
219997,32.462258,-92.160431,4.0,3
219998,35.557125,-82.535757,4.6,14
219999,42.229032,-87.940695,5.0,8


In [24]:
df_metadata_cat

,name,address,category
0,Neurocare & Family Services,"Neurocare & Family Services, 1749 S Naperville...",[Doctor]
1,Bayview Canvas,"Bayview Canvas, 22460 Glass Ln, Port Charlotte...",[Upholstery shop]
2,Peluquería Acapulco Barbershop,"Peluquería Acapulco Barbershop, 1014 Gault Ave...",[Barber shop]
3,Mountain Travelers Hike and Ski Shop,"Mountain Travelers Hike and Ski Shop, 147 US-4...","[Ski shop, Camping store, Car accessories store]"
4,Lovely Nails,"Lovely Nails, 636 Brandon Ave SW, Roanoke, VA ...",[Nail salon]
...,...,...,...
219996,Chase Bank,"Chase Bank, 5010 Elk Grove Blvd Ste #180, Elk ...","[Bank, ATM]"
219997,Lindsay Heating Cooling and Refrigeration,"Lindsay Heating Cooling and Refrigeration, 241...","[HVAC contractor, Air conditioning repair serv..."
219998,Fosnight Center for Sexual Health,"Fosnight Center for Sexual Health, 6 Yorkshire...","[Medical Center, Counselor, Medical clinic, Me..."
219999,Arbor Hills Dental Care,"Arbor Hills Dental Care, 175 N Milwaukee Ave #...","[Dental clinic, Dentist]"
